In [1]:
## import pandas as pd
import numpy as np
import tensorflow as tf
import os 
from glob import glob
import tqdm
from tqdm import tqdm
import pandas as pd
import pydicom
import nrrd
from einops import rearrange, reduce, repeat

In [2]:
import cv2
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import math
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from sklearn.model_selection import train_test_split

In [11]:
ptid = sorted(os.listdir('./data/original1125/'))
df_ptid = pd.DataFrame(ptid, columns=['ptid'])
all_inputpath = list('./data/original1125/'+df_ptid['ptid']+'/img/')
all_label = './data/original1125/'+df_ptid['ptid']
all_labelpath = []
for i in range(len(all_label)):
    all_labelpath.append(glob(all_label[i]+'/*.nrrd')[0])

In [12]:
len(df_ptid)

43

In [13]:
all_inputpath[0], all_labelpath[0]

('./data/original1125/09638755/img/',
 './data/original1125/09638755/Segmentation_10.seg.nrrd')

In [14]:
train_input_files = all_inputpath[:31]
train_label_files = all_labelpath[:31]

val_input_files = all_inputpath[31:38]
val_label_files = all_labelpath[31:38]

test_input_files = all_inputpath[38:]
test_label_files = all_labelpath[38:]

print(len(train_input_files), len(val_input_files), len(test_input_files))

31 7 5


In [ ]:
print(train_input_files)
print(val_input_files)
print(test_input_files)

In [ ]:
print(train_label_files)
print(val_label_files)
print(test_label_files)

In [ ]:
BATCH_SIZE = 8
img_size = 128
weights = None
learning_rate = 1e-4
EPOCHS = 400
dropout_rate = 0.1

In [ ]:
def cut_img(img_path_list, save_path, stride):
    os.makedirs(f'{save_path}{img_size}', exist_ok=True)
    num = 0
    for path in tqdm(img_path_list):
        l = len(os.listdir(path))
        print(l)
        rawimg = np.zeros((512,512,l))
        for i in range(l):
            slice = pydicom.read_file(path+'IMG'+str(i+1))
            s = int(slice.RescaleSlope)
            b = int(slice.RescaleIntercept)
            image = s * slice.pixel_array + b
            rawimg[:,:,i]=image
        rawimg = np.expand_dims(rawimg,axis=3)    
        clipimg = np.clip(rawimg, -1450,50)
        img = (clipimg-np.min(clipimg))/(np.max(clipimg)-np.min(clipimg))
        for top in range(0, img.shape[0], stride):
            for left in range(0, img.shape[1], stride):
                for slicer in range(0, img.shape[2], stride):
                    piece = np.zeros([img_size, img_size, img_size, 1])
                    temp = img[top:top+img_size, left:left+img_size, slicer:slicer+img_size, :]
                    piece[:temp.shape[0], :temp.shape[1], :temp.shape[2], :] = temp
                    np.save(f'{save_path}{img_size}/{num}.npy', piece)
                    num+=1

In [ ]:
def cut_img_label(img_path_list, save_path, stride):
    os.makedirs(f'{save_path}{img_size}', exist_ok=True)
    num = 0
    for path in tqdm(img_path_list):
        readdata, header = nrrd.read(path, index_order='C') #readdata (c h w)
        readdata = readdata[::-1] #reverse axial (-c h w)
        readdata = rearrange(readdata, 'c h w  -> h w c')
        readdata[readdata>0.5] = 1 #integrate label
        rawimg = np.expand_dims(readdata,axis=3)
        clipimg = np.clip(rawimg, -1450,50)
        img = (clipimg-np.min(clipimg))/(np.max(clipimg)-np.min(clipimg))
        for top in range(0, img.shape[0], stride):
            for left in range(0, img.shape[1], stride):
                for slicer in range(0, img.shape[2], stride):
                    piece = np.zeros([img_size, img_size, img_size, 1])
                    temp = img[top:top+img_size, left:left+img_size, slicer:slicer+img_size, :]
                    piece[:temp.shape[0], :temp.shape[1], :temp.shape[2], :] = temp
                    np.save(f'{save_path}{img_size}/{num}.npy', piece)
                    num+=1

In [ ]:
cut_img(train_input_files, f'data{img_size}/train_input_img_', img_size)
cut_img_label(train_label_files, f'data{img_size}/train_label_img_', img_size)
cut_img(val_input_files, f'data{img_size}/val_input_img_', img_size)
cut_img_label(val_label_files, f'data{img_size}/val_label_img_', img_size)